In [22]:
import random

def generate_graph(output_file="/kaggle/working/graph_10M.txt", num_nodes=10000000, num_edges=20000000):
    edges = set()
    with open(output_file, "w") as f:
        for i in range(num_nodes - 1):
            w = random.randint(1, 20)
            f.write(f"{i} {i + 1} {w}\n")
            edges.add((i, i + 1))

        while len(edges) < num_edges:
            u = random.randint(0, num_nodes - 1)
            v = random.randint(0, num_nodes - 1)
            if u != v and (u, v) not in edges:
                w = random.randint(1, 20)
                f.write(f"{u} {v} {w}\n")
                edges.add((u, v))

    print(f"Graph written: {len(edges)} edges to {output_file}")

generate_graph()


Graph written: 20000 edges to /kaggle/working/graph_10K.txt


In [19]:
!wc -l /kaggle/working/graph_10M.txt

20000000 /kaggle/working/graph_10M.txt


In [5]:
%%writefile /kaggle/working/dijkstra_speedup.cu
#include <iostream>
#include <vector>
#include <queue>
#include <fstream>
#include <climits>
#include <cuda.h>
#include <chrono>
#include <thread>
#include <mutex>
#include <condition_variable>
#include <atomic>
#include <iomanip>
#include <algorithm>  // Added this include for reverse() function

#define INF INT_MAX
#define MAX_NODES 10000000

using namespace std;

// Animation frames for terminal visualization
const vector<string> spinnerFrames = {"-", "\\", "|", "/"};

struct Edge {
    int u, v, w;
};

// Shared variables for progress display
atomic<bool> processing(true);
atomic<int> progress(0);
mutex mtx;
condition_variable cv;

// CUDA Kernel for Parallel Relaxation with path tracking
__global__ void relaxEdges(int *d_dist, int *d_edges, int *d_weights, int *d_offset, int *d_updated, int *d_prev, int V) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid >= V || d_updated[tid] == 0) return;

    d_updated[tid] = 0;

    for (int i = d_offset[tid]; i < d_offset[tid + 1]; i++) {
        int neighbor = d_edges[i];
        int weight = d_weights[i];

        if (d_dist[tid] != INF && d_dist[tid] + weight < d_dist[neighbor]) {
            atomicMin(&d_dist[neighbor], d_dist[tid] + weight);
            d_prev[neighbor] = tid;  // Record the predecessor
            d_updated[neighbor] = 1;
        }
    }
}

// CUDA Kernel for counting relaxations
__global__ void countUpdates(int *d_updated, int *d_count, int V) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid >= V) return;
    
    if(d_updated[tid] == 1) {
        atomicAdd(d_count, 1);
    }
}

// Function to print spinner animation
void animateProgress() {
    int frame = 0;
    
    while (processing.load()) {
        cout << "\r[";
        for (int i = 0; i < 50; i++) {
            if (i < progress.load() / 2) {
                cout << "=";
            } else if (i == progress.load() / 2) {
                cout << ">";
            } else {
                cout << " ";
            }
        }
        
        cout << "] " << progress.load() << "% " << spinnerFrames[frame] << " ";
        cout.flush();
        
        frame = (frame + 1) % spinnerFrames.size();
        this_thread::sleep_for(chrono::milliseconds(80));
    }
    
    // Final progress bar
    cout << "\r[";
    for (int i = 0; i < 50; i++) {
        cout << "=";
    }
    cout << "] 100% ✓" << endl;
}

// Sequential Dijkstra with path tracking
vector<int> dijkstraSequential(int src, vector<vector<pair<int, int>>> &adj, int V, vector<int> &prev) {
    vector<int> dist(V, INF);
    prev.resize(V, -1);
    priority_queue<pair<int, int>, vector<pair<int, int>>, greater<>> pq;
    dist[src] = 0;
    pq.emplace(0, src);

    int processedNodes = 0;
    
    while (!pq.empty()) {
        auto [d, u] = pq.top(); pq.pop();
        if (d > dist[u]) continue;
        
        processedNodes++;
        if (processedNodes % (V/100 + 1) == 0) {
            progress.store(min(99, (processedNodes * 100) / V));
        }

        for (auto &[v, w] : adj[u]) {
            if (dist[u] + w < dist[v]) {
                dist[v] = dist[u] + w;
                prev[v] = u;  // Store predecessor
                pq.emplace(dist[v], v);
            }
        }
    }
    
    return dist;
}

// Function to reconstruct path from source to target
vector<int> reconstructPath(const vector<int> &prev, int target) {
    vector<int> path;
    for (int at = target; at != -1; at = prev[at]) {
        path.push_back(at);
    }
    reverse(path.begin(), path.end());
    return path;
}

// Function to print path with edge weights in 1-(10)>2 format
void printPath(const vector<int> &path, vector<vector<pair<int, int>>> &adj, const vector<int> &dist) {
    if (path.empty()) {
        cout << "No path exists!" << endl;
        return;
    }
    
    cout << "\n========================================\n";
    cout << "     Shortest Path with Weights\n";
    cout << "========================================\n";
    
    cout << "Path: ";
    for (size_t i = 0; i < path.size(); i++) {
        cout << path[i];
        if (i < path.size() - 1) {
            // Find edge weight between path[i] and path[i+1]
            int u = path[i];
            int v = path[i+1];
            int weight = -1;
            for (auto &p : adj[u]) {
                if (p.first == v) {
                    weight = p.second;
                    break;
                }
            }
            cout << "-(" << weight << ")>";
        }
    }
    cout << "\nTotal Distance: " << dist[path.back()] << endl;
}

void readGraphFlat(const string &filename, vector<Edge> &edges, int &V, int &E) {
    ifstream infile(filename);
    int u, v, w;
    V = 0; E = 0;
    
    int totalBytes = 0;
    infile.seekg(0, ios::end);
    totalBytes = infile.tellg();
    infile.seekg(0, ios::beg);
    
    long long bytesRead = 0;
    int lastProgress = 0;
    
    while (infile >> u >> v >> w) {
        edges.push_back({u, v, w});
        V = max(V, max(u, v));
        E++;
        
        bytesRead = infile.tellg();
        int currentProgress = min(99, static_cast<int>(bytesRead * 100 / totalBytes));
        if (currentProgress > lastProgress) {
            progress.store(currentProgress);
            lastProgress = currentProgress;
        }
        
        if (E % 1000000 == 0) {
            cout << "\r" << E << " edges read... (" << currentProgress << "%)     ";
            cout.flush();
        }
    }
    V++;
    cout << "\r" << E << " edges read... (100%)     " << endl;
}

// Visualize the graph using ASCII art
void visualizeGraph(const vector<int> &dist, const vector<int> &prev, int V, int target) {
    // Find the path to target
    vector<int> path = reconstructPath(prev, target);
    
    cout << "\n========================================\n";
    cout << "     Graph Visualization (ASCII)\n";
    cout << "========================================\n";
    
    // Create an ASCII representation of the graph focusing on the path
    const int width = 80;
    const int height = 20;
    
    vector<vector<char>> grid(height, vector<char>(width, ' '));
    
    // Place the nodes in the grid
    for (int i = 0; i < min(V, 50); i++) {
        int x = i % width;
        int y = i / width % height;
        grid[y][x] = (dist[i] < INF) ? 'o' : '.';
    }
    
    // Mark the path nodes
    for (int node : path) {
        if (node < 50) {  // Only show first 50 nodes
            int x = node % width;
            int y = node / width % height;
            grid[y][x] = '*';
        }
    }
    
    // Display the grid
    for (const auto &row : grid) {
        for (char c : row) {
            cout << c;
        }
        cout << endl;
    }
    
    cout << "\nLegend: '*' = path node, 'o' = reachable node, '.' = unreachable node\n";
}

int main(int argc, char* argv[]) {
    if (argc < 2) {
        cout << "Usage: ./dijkstra <input_graph_file> [target_node]\n";
        return 1;
    }

    string filename = argv[1];
    int target = (argc > 2) ? atoi(argv[2]) : 999999;  // Default target node
    
    vector<Edge> edges;
    int V, E;

    cout << "\n========================================\n";
    cout << "     Reading Graph\n";
    cout << "========================================\n";
    
    // Start progress animation thread for graph reading
    thread animationThread(animateProgress);
    progress.store(0);
    
    readGraphFlat(filename, edges, V, E);
    
    // Build adjacency list for sequential
    vector<vector<pair<int, int>>> adj(V);
    for (const auto &e : edges) {
        adj[e.u].emplace_back(e.v, e.w);
    }
    
    cout << "\nGraph loaded with " << V << " nodes and " << E << " edges.\n";

    cout << "\n========================================\n";
    cout << "     Running Sequential Dijkstra\n";
    cout << "========================================\n";
    
    // Reset progress for sequential algorithm
    progress.store(0);
    
    vector<int> prev_seq;
    auto start_seq = chrono::high_resolution_clock::now();
    vector<int> dist_seq = dijkstraSequential(0, adj, V, prev_seq);
    auto end_seq = chrono::high_resolution_clock::now();
    double time_seq = chrono::duration<double, milli>(end_seq - start_seq).count();
    // Print sequential path and weights if target is valid
    if (target < V) {
        cout << "\n========================================\n";
        cout << "     Sequential Path to Target Node " << target << "\n";
        cout << "========================================\n";
        
        if (dist_seq[target] == INF) {
            cout << "No path exists to node " << target << endl;
        } else {
            cout << "Distance: " << dist_seq[target] << endl;
            vector<int> path_seq = reconstructPath(prev_seq, target);
            
            cout << "Path: ";
            for (size_t i = 0; i < path_seq.size(); i++) {
                cout << path_seq[i];
                if (i < path_seq.size() - 1) cout << " -> ";
            }
            cout << endl;
            
            printPath(path_seq, adj, dist_seq);
        }
    }

    // CSR-like format for CUDA
    vector<int> h_offset(V + 1), h_edges, h_weights;
    int edge_count = 0;
    for (int i = 0; i < V; i++) {
        h_offset[i] = edge_count;
        for (auto &[v, w] : adj[i]) {
            h_edges.push_back(v);
            h_weights.push_back(w);
            edge_count++;
        }
    }
    h_offset[V] = edge_count;

    int *d_edges, *d_weights, *d_offset, *d_dist, *d_updated, *d_prev, *d_count;

    cudaMalloc(&d_edges, h_edges.size() * sizeof(int));
    cudaMalloc(&d_weights, h_weights.size() * sizeof(int));
    cudaMalloc(&d_offset, (V + 1) * sizeof(int));
    cudaMalloc(&d_dist, V * sizeof(int));
    cudaMalloc(&d_updated, V * sizeof(int));
    cudaMalloc(&d_prev, V * sizeof(int));
    cudaMalloc(&d_count, sizeof(int));

    vector<int> h_dist(V, INF); h_dist[0] = 0;
    vector<int> h_updated(V, 0); h_updated[0] = 1;
    vector<int> h_prev(V, -1);
    int h_count = 0;

    cudaMemcpy(d_edges, h_edges.data(), h_edges.size() * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_weights, h_weights.data(), h_weights.size() * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_offset, h_offset.data(), (V + 1) * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_dist, h_dist.data(), V * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_updated, h_updated.data(), V * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_prev, h_prev.data(), V * sizeof(int), cudaMemcpyHostToDevice);

    cout << "\n========================================\n";
    cout << "     Running Parallel CUDA Relaxation\n";
    cout << "========================================\n";
    
    // Reset progress for parallel algorithm
    progress.store(0);
    
    auto start_par = chrono::high_resolution_clock::now();
    
    int iterations = 0;
    bool hasUpdates = true;
    
    while (hasUpdates && iterations < V - 1) {
        iterations++;
        
        // Zero the count
        h_count = 0;
        cudaMemcpy(d_count, &h_count, sizeof(int), cudaMemcpyHostToDevice);
        
        // Launch relaxation kernel
        relaxEdges<<<(V + 255) / 256, 256>>>(d_dist, d_edges, d_weights, d_offset, d_updated, d_prev, V);
        cudaDeviceSynchronize();
        
        // Count updates
        countUpdates<<<(V + 255) / 256, 256>>>(d_updated, d_count, V);
        cudaDeviceSynchronize();
        
        // Check if we have updates
        cudaMemcpy(&h_count, d_count, sizeof(int), cudaMemcpyDeviceToHost);
        hasUpdates = (h_count > 0);
        
        // Update progress
        progress.store(min(99, (iterations * 100) / V));
    }

    auto end_par = chrono::high_resolution_clock::now();
    double time_par = chrono::duration<double, milli>(end_par - start_par).count();

    cudaMemcpy(h_dist.data(), d_dist, V * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_prev.data(), d_prev, V * sizeof(int), cudaMemcpyDeviceToHost);
    
    // Signal animation thread to stop
    processing.store(false);
    animationThread.join();

    cout << "\n========================================\n";
    cout << "     Results\n";
    cout << "========================================\n";
    cout << "Sequential Time: " << time_seq << " ms\n";
    cout << "Parallel Time:   " << time_par << " ms\n";
    cout << "Speedup:         " << time_seq / time_par << "x\n";
    cout << "CUDA Iterations: " << iterations << endl;
    
    // Verify results match
    bool match = true;
    for (int i = 0; i < min(V, 1000); i++) {  // Check first 1000 nodes
        if (dist_seq[i] != h_dist[i] && !(dist_seq[i] == INF && h_dist[i] == INF)) {
            cout << "Mismatch at node " << i << ": Sequential = " << dist_seq[i] << ", Parallel = " << h_dist[i] << endl;
            match = false;
            break;
        }
    }
    
    if (match) {
        cout << "Results match! ✓\n";
    } else {
        cout << "Results differ! ✗\n";
    }
    
    // Print path to target node
    if (target < V) {
        cout << "\n========================================\n";
        cout << "     Path to Target Node " << target << "\n";
        cout << "========================================\n";
        
        if (h_dist[target] == INF) {
            cout << "No path exists to node " << target << endl;
        } else {
            cout << "Distance: " << h_dist[target] << endl;
            
            // Reconstruct path from source to target
            vector<int> path = reconstructPath(h_prev, target);
            
            cout << "Path: ";
            for (size_t i = 0; i < path.size(); i++) {
                cout << path[i];
                if (i < path.size() - 1) cout << " -> ";
            }
            cout << endl;
            
            // Detailed path with weights
            printPath(path, adj, h_dist);
            
            // ASCII visualization of the graph
            visualizeGraph(h_dist, h_prev, V, target);
        }
    }

    cudaFree(d_edges);
    cudaFree(d_weights);
    cudaFree(d_offset);
    cudaFree(d_dist);
    cudaFree(d_updated);
    cudaFree(d_prev);
    cudaFree(d_count);

    return 0;
}

Overwriting /kaggle/working/dijkstra_speedup.cu


In [6]:
!nvcc /kaggle/working/dijkstra_speedup.cu -o /kaggle/working/dijkstra

In [23]:
!/kaggle/working/dijkstra /kaggle/working/graph_10M.txt 999999


     Reading Graph
20000 edges read... (100%)                         ] 0% - 

Graph loaded with 10000 nodes and 20000 edges.

     Running Sequential Dijkstra

     Sequential Path to Target Node 999
Distance: 130
Path: 0 -> 1955 -> 1956 -> 5069 -> 1357 -> 2434 -> 2435 -> 7241 -> 9202 -> 9203 -> 9204 -> 997 -> 998 -> 999

     Shortest Path with Weights
Path: 0-(13)>1955-(20)>1956-(4)>5069-(5)>1357-(4)>2434-(14)>2435-(5)>7241-(6)>9202-(3)>9203-(15)>9204-(13)>997-(8)>998-(20)>999
Total Distance: 130
[=================================================>] 99% \ 
     Running Parallel CUDA Relaxation
[==================================================] 100% ✓

     Results
Sequential Time: 13.2493 ms
Parallel Time:   1.11473 ms
Speedup:         11.8857x
CUDA Iterations: 29
Results match! ✓

     Path to Target Node 999
Distance: 130
Path: 0 -> 1955 -> 1956 -> 5069 -> 1357 -> 2434 -> 2435 -> 7241 -> 9202 -> 9203 -> 9204 -> 997 -> 998 -> 999

     Shortest Path with Weights
Path: 0-(13)>1955

In [ ]:
# %%writefile /kaggle/working/parallel.cu

In [9]:
%%writefile /kaggle/working/parallel.cu
#include <iostream>
#include <vector>
#include <fstream>
#include <climits>
#include <cuda.h>
#include <chrono>
#include <thread>
#include <mutex>
#include <condition_variable>
#include <atomic>
#include <algorithm>
#include <unordered_map>
#include <iomanip>  // Added for setprecision

#define INF INT_MAX
#define MAX_NODES 10000000

using namespace std;

// Animation frames for terminal visualization
const vector<string> spinnerFrames = {"-", "\\", "|", "/"};

// Shared variables for progress display
atomic<bool> processing(true);
atomic<int> progress(0);
mutex mtx;
condition_variable cv;

// Function to print spinner animation
void animateProgress() {
    int frame = 0;
    
    while (processing.load()) {
        cout << "\r[";
        for (int i = 0; i < 50; i++) {
            if (i < progress.load() / 2) {
                cout << "=";
            } else if (i == progress.load() / 2) {
                cout << ">";
            } else {
                cout << " ";
            }
        }
        
        cout << "] " << progress.load() << "% " << spinnerFrames[frame] << " ";
        cout.flush();
        
        frame = (frame + 1) % spinnerFrames.size();
        this_thread::sleep_for(chrono::milliseconds(80));
    }
    
    // Final progress bar
    cout << "\r[";
    for (int i = 0; i < 50; i++) {
        cout << "=";
    }
    cout << "] 100% ✓" << endl;
}

// CUDA Kernel for Parallel Relaxation with path tracking
__global__ void relaxEdges(int *d_dist, int *d_edges, int *d_weights, int *d_offset, int *d_updated, int *d_prev, int V) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid >= V || d_updated[tid] == 0) return;

    d_updated[tid] = 0;

    for (int i = d_offset[tid]; i < d_offset[tid + 1]; i++) {
        int neighbor = d_edges[i];
        int weight = d_weights[i];
        
        // Check for valid neighbor
        if (neighbor < 0 || neighbor >= V) continue;
        
        // Safe distance update that avoids overflow
        if (d_dist[tid] != INF && 
            d_dist[tid] < INF - weight && // Prevent overflow
            d_dist[tid] + weight < d_dist[neighbor]) {
            
            atomicMin(&d_dist[neighbor], d_dist[tid] + weight);
            d_prev[neighbor] = tid;  // Record the predecessor
            d_updated[neighbor] = 1;
        }
    }
}

// CUDA Kernel for counting relaxations
__global__ void countUpdates(int *d_updated, int *d_count, int V) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid >= V) return;
    
    if(d_updated[tid] == 1) {
        atomicAdd(d_count, 1);
    }
}

// Function to directly load graph to GPU memory in chunks
bool loadGraphDirectlyToGPU(const string &filename, int &V, int &E, 
                          int **d_edges, int **d_weights, int **d_offset,
                          vector<int> &h_offset) {  // Added h_offset as output parameter
    ifstream infile(filename);
    if (!infile) {
        cerr << "Error opening file: " << filename << endl;
        return false;
    }
    
    cout << "Counting nodes and edges...\n";
    
    // First pass: count nodes and edges
    int u, v, w;
    V = 0;
    E = 0;
    
    // Get file size for progress reporting
    infile.seekg(0, ios::end);
    size_t fileSize = infile.tellg();
    infile.seekg(0, ios::beg);
    
    size_t bytesRead = 0;
    unordered_map<int, bool> nodeMap; // Track unique nodes
    
    while (infile >> u >> v >> w) {
        // Track max node ID
        V = max(V, max(u, v) + 1);
        
        // Track unique nodes
        nodeMap[u] = true;
        nodeMap[v] = true;
        
        E++;
        
        // Update progress every 100,000 edges
        if (E % 100000 == 0) {
            bytesRead = infile.tellg();
            progress.store(min(49, static_cast<int>(bytesRead * 100 / fileSize)));
            cout << "\rCounting: " << E << " edges found... (" << progress.load() << "%)     ";
            cout.flush();
        }
        
        // Validate edge weights
        if (w <= 0) {
            cout << "\nWarning: Edge " << u << " -> " << v << " has non-positive weight: " << w << endl;
            cout << "Replacing with weight = 1" << endl;
            w = 1; // Use positive weight for SSSP
        }
    }
    
    cout << "\nFound " << V << " nodes and " << E << " edges (" << nodeMap.size() << " unique nodes).\n";
    cout << "Allocating memory...\n";
    
    // Allocate memory for CSR format on host temporarily
    h_offset.resize(V + 1, 0);  // Using the reference parameter
    vector<vector<pair<int, int>>> tempAdj(V);
    
    // Reset file for second pass
    infile.clear();
    infile.seekg(0, ios::beg);
    
    // Second pass: build temporary adjacency list
    cout << "Building adjacency list...\n";
    progress.store(50);
    
    int edgesRead = 0;
    while (infile >> u >> v >> w) {
        // Ensure positive weight
        if (w <= 0) w = 1;
        
        if (u < V && v < V) {
            tempAdj[u].emplace_back(v, w);
            edgesRead++;
            
            if (edgesRead % 100000 == 0) {
                progress.store(50 + min(24, static_cast<int>(edgesRead * 25 / E)));
            }
        }
    }
    
    // Build CSR offset array
    cout << "Building CSR format...\n";
    int edge_count = 0;
    for (int i = 0; i < V; i++) {
        h_offset[i] = edge_count;
        edge_count += tempAdj[i].size();
    }
    h_offset[V] = edge_count;
    
    // Allocate device memory
    cudaMalloc(d_offset, (V + 1) * sizeof(int));
    cudaMalloc(d_edges, E * sizeof(int));
    cudaMalloc(d_weights, E * sizeof(int));
    
    // Copy offset array to device
    cudaMemcpy(*d_offset, h_offset.data(), (V + 1) * sizeof(int), cudaMemcpyHostToDevice);
    
    // Allocate temporary host arrays for edges and weights
    vector<int> h_edges(E);
    vector<int> h_weights(E);
    
    // Fill edge and weight arrays
    edge_count = 0;
    for (int i = 0; i < V; i++) {
        for (auto &p : tempAdj[i]) {
            h_edges[edge_count] = p.first;
            h_weights[edge_count] = p.second;
            edge_count++;
        }
        
        // Update progress
        if (i % (V/100 + 1) == 0) {
            progress.store(75 + min(24, static_cast<int>(i * 25 / V)));
        }
    }
    
    // Copy edges and weights to device in chunks to save memory
    const int CHUNK_SIZE = 5000000; // Adjust based on available memory
    for (int i = 0; i < E; i += CHUNK_SIZE) {
        int size = min(CHUNK_SIZE, E - i);
        cudaMemcpy(*d_edges + i, h_edges.data() + i, size * sizeof(int), cudaMemcpyHostToDevice);
        cudaMemcpy(*d_weights + i, h_weights.data() + i, size * sizeof(int), cudaMemcpyHostToDevice);
    }
    
    progress.store(99);
    cout << "Graph loaded to GPU successfully.\n";
    
    return true;
}

// Function to reconstruct path from source to target
vector<int> reconstructPath(const vector<int> &prev, int target) {
    vector<int> path;
    for (int at = target; at != -1; at = prev[at]) {
        path.push_back(at);
        // Guard against cycles
        if (path.size() > 1000000) {
            cout << "Warning: Path reconstruction exceeded 1,000,000 steps, possible cycle detected." << endl;
            break;
        }
    }
    reverse(path.begin(), path.end());
    return path;
}

// Function to check CUDA errors
void checkCudaError(cudaError_t err, const char *msg) {
    if (err != cudaSuccess) {
        cerr << "CUDA Error: " << msg << " - " << cudaGetErrorString(err) << endl;
        exit(EXIT_FAILURE);
    }
}

int main(int argc, char* argv[]) {
    if (argc < 2) {
        cout << "Usage: ./dijkstra <input_graph_file> [target_node] [source_node]\n";
        return 1;
    }

    string filename = argv[1];
    int target = (argc > 2) ? atoi(argv[2]) : 999999;  // Default target node
    int source = (argc > 3) ? atoi(argv[3]) : 0;       // Default source node
    
    int V, E;

    cout << "\n========================================\n";
    cout << "     Loading Graph Directly to GPU\n";
    cout << "========================================\n";
    
    // Start progress animation thread for graph loading
    thread animationThread(animateProgress);
    progress.store(0);
    
    int *d_edges, *d_weights, *d_offset;
    vector<int> h_offset;  // This is now used to keep the host copy of offset array
    if (!loadGraphDirectlyToGPU(filename, V, E, &d_edges, &d_weights, &d_offset, h_offset)) {
        processing.store(false);
        animationThread.join();
        return 1;
    }
    
    // Verify source and target are valid
    if (source < 0 || source >= V) {
        cout << "Invalid source node " << source << ". Must be between 0 and " << (V-1) << endl;
        source = 0;
        cout << "Using default source node 0 instead." << endl;
    }
    
    if (target < 0 || target >= V) {
        cout << "Invalid target node " << target << ". Must be between 0 and " << (V-1) << endl;
        target = min(V-1, 999999);
        cout << "Using target node " << target << " instead." << endl;
    }
    
    // Allocate device memory for distance, updates and predecessor arrays
    int *d_dist, *d_updated, *d_prev, *d_count;
    cudaError_t err;
    
    err = cudaMalloc(&d_dist, V * sizeof(int));
    checkCudaError(err, "Failed to allocate device memory for distances");
    
    err = cudaMalloc(&d_updated, V * sizeof(int));
    checkCudaError(err, "Failed to allocate device memory for updates");
    
    err = cudaMalloc(&d_prev, V * sizeof(int));
    checkCudaError(err, "Failed to allocate device memory for predecessors");
    
    err = cudaMalloc(&d_count, sizeof(int));
    checkCudaError(err, "Failed to allocate device memory for counter");
    
    // Initialize distance array (all INF except source = 0)
    vector<int> h_dist(V, INF);
    h_dist[source] = 0;
    err = cudaMemcpy(d_dist, h_dist.data(), V * sizeof(int), cudaMemcpyHostToDevice);
    checkCudaError(err, "Failed to copy initial distances to device");
    
    // Initialize updated array (only source is updated initially)
    vector<int> h_updated(V, 0);
    h_updated[source] = 1;
    err = cudaMemcpy(d_updated, h_updated.data(), V * sizeof(int), cudaMemcpyHostToDevice);
    checkCudaError(err, "Failed to copy initial updates to device");
    
    // Initialize predecessor array (all -1)
    vector<int> h_prev(V, -1);
    err = cudaMemcpy(d_prev, h_prev.data(), V * sizeof(int), cudaMemcpyHostToDevice);
    checkCudaError(err, "Failed to copy initial predecessors to device");
    
    cout << "\n========================================\n";
    cout << "     Running Parallel CUDA Dijkstra\n";
    cout << "========================================\n";
    
    // Reset progress for parallel algorithm
    progress.store(0);
    
    // Create CUDA events for timing
    cudaEvent_t start_event, stop_event;
    cudaEventCreate(&start_event);
    cudaEventCreate(&stop_event);
    
    // Start timing
    cudaEventRecord(start_event);
    
    int iterations = 0;
    bool hasUpdates = true;
    int h_count = 0;
    int max_iterations = min(V, 10000); // Limit iterations to avoid infinite loops
    
    while (hasUpdates && iterations < max_iterations) {
        iterations++;
        
        // Reset count
        h_count = 0;
        err = cudaMemcpy(d_count, &h_count, sizeof(int), cudaMemcpyHostToDevice);
        checkCudaError(err, "Failed to reset update counter");
        
        // Launch relaxation kernel
        int blockSize = 256;
        int numBlocks = (V + blockSize - 1) / blockSize;
        relaxEdges<<<numBlocks, blockSize>>>(d_dist, d_edges, d_weights, d_offset, d_updated, d_prev, V);
        err = cudaGetLastError();
        checkCudaError(err, "Failed to launch relaxation kernel");
        
        // Count updates
        countUpdates<<<numBlocks, blockSize>>>(d_updated, d_count, V);
        err = cudaGetLastError();
        checkCudaError(err, "Failed to launch count updates kernel");
        
        // Check if we have updates
        err = cudaMemcpy(&h_count, d_count, sizeof(int), cudaMemcpyDeviceToHost);
        checkCudaError(err, "Failed to copy update count from device");
        hasUpdates = (h_count > 0);
        
        // Update progress (scale iterations to percentage)
        progress.store(min(99, (iterations * 100) / min(V, 1000)));
        
        // Print update every 1000 iterations
        if (iterations % 1000 == 0) {
            cout << "\rIteration " << iterations << ", updates: " << h_count << "     ";
            cout.flush();
        }
    }
    
    // Stop timing
    cudaEventRecord(stop_event);
    cudaEventSynchronize(stop_event);
    
    // Calculate execution time
    float kernel_time_ms = 0;
    cudaEventElapsedTime(&kernel_time_ms, start_event, stop_event);
    
    // Copy results back
    err = cudaMemcpy(h_dist.data(), d_dist, V * sizeof(int), cudaMemcpyDeviceToHost);
    checkCudaError(err, "Failed to copy distances from device");
    
    err = cudaMemcpy(h_prev.data(), d_prev, V * sizeof(int), cudaMemcpyDeviceToHost);
    checkCudaError(err, "Failed to copy predecessors from device");
    
    // Signal animation thread to stop
    processing.store(false);
    animationThread.join();

    cout << "\n========================================\n";
    cout << "     Results\n";
    cout << "========================================\n";
    cout << "CUDA Kernel Time:            " << kernel_time_ms << " ms\n";
    cout << "CUDA Iterations:             " << iterations << endl;
    
    // Print info about unreachable nodes
    int unreachable = 0;
    for (int i = 0; i < V; i++) {
        if (h_dist[i] == INF) unreachable++;
    }
    cout << "Unreachable nodes:           " << unreachable << " (" 
         << fixed << setprecision(2) << (unreachable * 100.0 / V) << "%)" << endl;
    
    // Print path to target node
    if (target < V) {
        cout << "\n========================================\n";
        cout << "     Path from Node " << source << " to Target Node " << target << "\n";
        cout << "========================================\n";
        
        if (h_dist[target] == INF) {
            cout << "No path exists to node " << target << endl;
        } else {
            cout << "Distance: " << h_dist[target] << endl;
            
            // Reconstruct path from source to target
            vector<int> path = reconstructPath(h_prev, target);
            
            cout << "Path: ";
            for (size_t i = 0; i < path.size(); i++) {
                cout << path[i];
                if (i < path.size() - 1) cout << " -> ";
            }
            cout << endl;
            
            // Display path length
            cout << "Path length: " << path.size() << " nodes" << endl;
            
            // Validate path by calculating distance manually
            int calculated_dist = 0;
            bool path_valid = true;
            
            // Get device edges and weights to host for validation
            vector<int> h_edges(E);
            vector<int> h_weights(E);
            cudaMemcpy(h_edges.data(), d_edges, E * sizeof(int), cudaMemcpyDeviceToHost);
            cudaMemcpy(h_weights.data(), d_weights, E * sizeof(int), cudaMemcpyDeviceToHost);
            
            for (size_t i = 0; i < path.size() - 1; i++) {
                int u = path[i];
                int v = path[i+1];
                bool edge_found = false;
                int edge_weight = -1;
                
                // Find edge weight in adjacency list
                for (int j = h_offset[u]; j < h_offset[u+1]; j++) {
                    if (h_edges[j] == v) {
                        edge_weight = h_weights[j];
                        edge_found = true;
                        break;
                    }
                }
                
                if (!edge_found) {
                    cout << "Error: Path contains invalid edge " << u << " -> " << v << endl;
                    path_valid = false;
                    break;
                }
                
                calculated_dist += edge_weight;
            }
            
            if (path_valid) {
                cout << "Calculated distance: " << calculated_dist 
                     << " (should match distance: " << h_dist[target] << ")" << endl;
                
                if (calculated_dist != h_dist[target]) {
                    cout << "Warning: Path distance doesn't match calculated shortest path distance!" << endl;
                }
            }
        }
    }

    // Cleanup
    cudaFree(d_edges);
    cudaFree(d_weights);
    cudaFree(d_offset);
    cudaFree(d_dist);
    cudaFree(d_updated);
    cudaFree(d_prev);
    cudaFree(d_count);
    cudaEventDestroy(start_event);
    cudaEventDestroy(stop_event);

    return 0;
}

Writing /kaggle/working/parallel.cu


In [10]:
!nvcc /kaggle/working/parallel.cu -o /kaggle/working/parallel

In [20]:
!/kaggle/working/parallel /kaggle/working/graph_10M.txt 999999


     Loading Graph Directly to GPU
[>                                                 ] Counting nodes and edges...
Counting: 2000000 edges found... (49%)             ] 49% | 
Found 1000000 nodes and 2000000 edges (1000000 unique nodes).
Allocating memory...
Building adjacency list...
[====================================>             ] 73% \ Building CSR format...
[===============================================>  ] 94% \ Graph loaded to GPU successfully.
[=================================================>] 99% \ 
     Running Parallel CUDA Dijkstra
[==================================================] 100% ✓

     Results
CUDA Kernel Time:            3.79264 ms
CUDA Iterations:             39
Unreachable nodes:           0 (0.00%)

     Path from Node 0 to Target Node 9999
Distance: 195
Path: 0 -> 1 -> 960854 -> 960855 -> 960856 -> 669661 -> 360805 -> 360806 -> 360807 -> 166249 -> 166250 -> 33474 -> 33475 -> 852883 -> 852884 -> 852885 -> 852886 -> 122858 -> 9995 -> 9996 -> 9997 -> 99

In [ ]:
# %%writefile /kaggle/working/sequential.cu

In [13]:
%%writefile /kaggle/working/sequential.cu
#include <iostream>
#include <vector>
#include <fstream>
#include <climits>
#include <chrono>
#include <thread>
#include <mutex>
#include <condition_variable>
#include <atomic>
#include <algorithm>
#include <unordered_map>
#include <iomanip>
#include <queue>  // Added for priority queue

#define INF INT_MAX
#define MAX_NODES 10000000

using namespace std;

// Animation frames for terminal visualization
const vector<string> spinnerFrames = {"-", "\\", "|", "/"};

// Shared variables for progress display
atomic<bool> processing(true);
atomic<int> progress(0);
mutex mtx;
condition_variable cv;

// Function to print spinner animation
void animateProgress() {
    int frame = 0;
    
    while (processing.load()) {
        cout << "\r[";
        for (int i = 0; i < 50; i++) {
            if (i < progress.load() / 2) {
                cout << "=";
            } else if (i == progress.load() / 2) {
                cout << ">";
            } else {
                cout << " ";
            }
        }
        
        cout << "] " << progress.load() << "% " << spinnerFrames[frame] << " ";
        cout.flush();
        
        frame = (frame + 1) % spinnerFrames.size();
        this_thread::sleep_for(chrono::milliseconds(80));
    }
    
    // Final progress bar
    cout << "\r[";
    for (int i = 0; i < 50; i++) {
        cout << "=";
    }
    cout << "] 100% ✓" << endl;
}

// Function to directly load graph to memory
bool loadGraph(const string &filename, int &V, int &E, 
               vector<vector<pair<int, int>>> &adj) {
    ifstream infile(filename);
    if (!infile) {
        cerr << "Error opening file: " << filename << endl;
        return false;
    }
    
    cout << "Counting nodes and edges...\n";
    
    // First pass: count nodes and edges
    int u, v, w;
    V = 0;
    E = 0;
    
    // Get file size for progress reporting
    infile.seekg(0, ios::end);
    size_t fileSize = infile.tellg();
    infile.seekg(0, ios::beg);
    
    size_t bytesRead = 0;
    unordered_map<int, bool> nodeMap; // Track unique nodes
    
    while (infile >> u >> v >> w) {
        // Track max node ID
        V = max(V, max(u, v) + 1);
        
        // Track unique nodes
        nodeMap[u] = true;
        nodeMap[v] = true;
        
        E++;
        
        // Update progress every 100,000 edges
        if (E % 100000 == 0) {
            bytesRead = infile.tellg();
            progress.store(min(49, static_cast<int>(bytesRead * 100 / fileSize)));
            cout << "\rCounting: " << E << " edges found... (" << progress.load() << "%)     ";
            cout.flush();
        }
        
        // Validate edge weights
        if (w <= 0) {
            cout << "\nWarning: Edge " << u << " -> " << v << " has non-positive weight: " << w << endl;
            cout << "Replacing with weight = 1" << endl;
            w = 1; // Use positive weight for SSSP
        }
    }
    
    cout << "\nFound " << V << " nodes and " << E << " edges (" << nodeMap.size() << " unique nodes).\n";
    cout << "Allocating memory...\n";
    
    // Allocate memory for adjacency list
    adj.resize(V);
    
    // Reset file for second pass
    infile.clear();
    infile.seekg(0, ios::beg);
    
    // Second pass: build adjacency list
    cout << "Building adjacency list...\n";
    progress.store(50);
    
    int edgesRead = 0;
    while (infile >> u >> v >> w) {
        // Ensure positive weight
        if (w <= 0) w = 1;
        
        if (u < V && v < V) {
            adj[u].emplace_back(v, w);
            edgesRead++;
            
            if (edgesRead % 100000 == 0) {
                progress.store(50 + min(49, static_cast<int>(edgesRead * 50 / E)));
            }
        }
    }
    
    cout << "Graph loaded successfully.\n";
    return true;
}

// Function to run sequential Dijkstra's algorithm
void sequentialDijkstra(const vector<vector<pair<int, int>>> &adj, vector<int> &dist, vector<int> &prev, int source, int V) {
    // Priority queue to store pairs of (distance, vertex)
    // Using greater for min-heap behavior
    priority_queue<pair<int, int>, vector<pair<int, int>>, greater<pair<int, int>>> pq;
    
    // Initialize distance and prev arrays
    dist.assign(V, INF);
    prev.assign(V, -1);
    
    // Distance to source is 0
    dist[source] = 0;
    pq.push({0, source});
    
    int iterations = 0;
    int vertices_processed = 0;
    
    while (!pq.empty()) {
        iterations++;
        
        // Get vertex with minimum distance
        int u = pq.top().second;
        int dist_u = pq.top().first;
        pq.pop();
        
        // Skip if we've found a better path already
        if (dist_u > dist[u]) continue;
        
        vertices_processed++;
        
        // Process all neighbors of u
        for (const auto &edge : adj[u]) {
            int v = edge.first;
            int weight = edge.second;
            
            // Relaxation step
            if (dist[u] != INF && dist[u] + weight < dist[v]) {
                dist[v] = dist[u] + weight;
                prev[v] = u;
                pq.push({dist[v], v});
            }
        }
        
        // Update progress (scale processed vertices to percentage)
        if (iterations % 1000 == 0) {
            progress.store(min(99, (vertices_processed * 100) / min(V, 100000)));
            cout << "\rProcessed " << vertices_processed << " vertices     ";
            cout.flush();
        }
    }
}

// Function to reconstruct path from source to target
vector<int> reconstructPath(const vector<int> &prev, int target) {
    vector<int> path;
    for (int at = target; at != -1; at = prev[at]) {
        path.push_back(at);
        // Guard against cycles
        if (path.size() > 1000000) {
            cout << "Warning: Path reconstruction exceeded 1,000,000 steps, possible cycle detected." << endl;
            break;
        }
    }
    reverse(path.begin(), path.end());
    return path;
}

int main(int argc, char* argv[]) {
    if (argc < 2) {
        cout << "Usage: ./dijkstra <input_graph_file> [target_node] [source_node]\n";
        return 1;
    }

    string filename = argv[1];
    int target = (argc > 2) ? atoi(argv[2]) : 999999;  // Default target node
    int source = (argc > 3) ? atoi(argv[3]) : 0;       // Default source node
    
    int V, E;
    vector<vector<pair<int, int>>> adj;

    cout << "\n========================================\n";
    cout << "     Loading Graph\n";
    cout << "========================================\n";
    
    // Start progress animation thread for graph loading
    thread animationThread(animateProgress);
    progress.store(0);
    
    if (!loadGraph(filename, V, E, adj)) {
        processing.store(false);
        animationThread.join();
        return 1;
    }
    
    // Verify source and target are valid
    if (source < 0 || source >= V) {
        cout << "Invalid source node " << source << ". Must be between 0 and " << (V-1) << endl;
        source = 0;
        cout << "Using default source node 0 instead." << endl;
    }
    
    if (target < 0 || target >= V) {
        cout << "Invalid target node " << target << ". Must be between 0 and " << (V-1) << endl;
        target = min(V-1, 999999);
        cout << "Using target node " << target << " instead." << endl;
    }
    
    // Vectors to store results
    vector<int> dist;
    vector<int> prev;
    
    cout << "\n========================================\n";
    cout << "     Running Sequential Dijkstra\n";
    cout << "========================================\n";
    
    // Reset progress for sequential algorithm
    progress.store(0);
    
    // Measure execution time
    auto start_time = chrono::high_resolution_clock::now();
    
    // Run sequential Dijkstra
    sequentialDijkstra(adj, dist, prev, source, V);
    
    auto end_time = chrono::high_resolution_clock::now();
    auto duration = chrono::duration_cast<chrono::milliseconds>(end_time - start_time);
    
    // Signal animation thread to stop
    processing.store(false);
    animationThread.join();

    cout << "\n========================================\n";
    cout << "     Results\n";
    cout << "========================================\n";
    cout << "Sequential Dijkstra Time:     " << duration.count() << " ms\n";
    
    // Print info about unreachable nodes
    int unreachable = 0;
    for (int i = 0; i < V; i++) {
        if (dist[i] == INF) unreachable++;
    }
    cout << "Unreachable nodes:           " << unreachable << " (" 
         << fixed << setprecision(2) << (unreachable * 100.0 / V) << "%)" << endl;
    
    // Print path to target node
    if (target < V) {
        cout << "\n========================================\n";
        cout << "     Path from Node " << source << " to Target Node " << target << "\n";
        cout << "========================================\n";
        
        if (dist[target] == INF) {
            cout << "No path exists to node " << target << endl;
        } else {
            cout << "Distance: " << dist[target] << endl;
            
            // Reconstruct path from source to target
            vector<int> path = reconstructPath(prev, target);
            
            cout << "Path: ";
            for (size_t i = 0; i < path.size(); i++) {
                cout << path[i];
                if (i < path.size() - 1) cout << " -> ";
            }
            cout << endl;
            
            // Display path length
            cout << "Path length: " << path.size() << " nodes" << endl;
            
            // Validate path by calculating distance manually
            int calculated_dist = 0;
            bool path_valid = true;
            
            for (size_t i = 0; i < path.size() - 1; i++) {
                int u = path[i];
                int v = path[i+1];
                bool edge_found = false;
                int edge_weight = -1;
                
                // Find edge weight in adjacency list
                for (const auto &edge : adj[u]) {
                    if (edge.first == v) {
                        edge_weight = edge.second;
                        edge_found = true;
                        break;
                    }
                }
                
                if (!edge_found) {
                    cout << "Error: Path contains invalid edge " << u << " -> " << v << endl;
                    path_valid = false;
                    break;
                }
                
                calculated_dist += edge_weight;
            }
            
            if (path_valid) {
                cout << "Calculated distance: " << calculated_dist 
                     << " (should match distance: " << dist[target] << ")" << endl;
                
                if (calculated_dist != dist[target]) {
                    cout << "Warning: Path distance doesn't match calculated shortest path distance!" << endl;
                }
            }
        }
    }

    return 0;
}

Writing /kaggle/working/sequential.cu


In [14]:
!nvcc /kaggle/working/sequential.cu -o /kaggle/working/sequential

In [21]:
!/kaggle/working/sequential /kaggle/working/graph_10M.txt 99999


     Loading Graph
[>                                                 ] Counting nodes and edges...
Counting: 2000000 edges found... (49%)             ] 49% \ 
Found 1000000 nodes and 2000000 edges (1000000 unique nodes).
Allocating memory...
[========================>                         ] 49% | Building adjacency list...
[================================================> ] 97% | Graph loaded successfully.

     Running Sequential Dijkstra
[==================================================] 100% ✓

     Results
Sequential Dijkstra Time:     2221 ms
Unreachable nodes:           0 (0.00%)

     Path from Node 0 to Target Node 9999
Distance: 195
Path: 0 -> 1 -> 960854 -> 960855 -> 960856 -> 669661 -> 360805 -> 360806 -> 360807 -> 166249 -> 166250 -> 33474 -> 33475 -> 852883 -> 852884 -> 852885 -> 852886 -> 122858 -> 9995 -> 9996 -> 9997 -> 9998 -> 9999
Path length: 23 nodes
Calculated distance: 195 (should match distance: 195)


In [24]:
%%writefile /kaggle/working/tp.cu
#include <algorithm> // Added this include for reverse() function
#include <atomic>
#include <chrono>
#include <climits>
#include <condition_variable>
#include <cuda.h>
#include <fstream>
#include <iomanip>
#include <iostream>
#include <mutex>
#include <queue>
#include <thread>
#include <vector>

#define INF INT_MAX
#define MAX_NODES 10000000
#define BLOCK_SIZE 256

using namespace std;

// Animation frames for terminal visualization
const vector<string> spinnerFrames = {"-", "\\", "|", "/"};

struct Edge {
  int u, v, w;
};

// Shared variables for progress display
atomic<bool> processing(true);
atomic<int> progress(0);
mutex mtx;
condition_variable cv;

// CUDA Kernel for Parallel Relaxation with path tracking and shared memory
__global__ void relaxEdges(int *d_dist, int *d_edges, int *d_weights,
                           int *d_offset, int *d_updated, int *d_prev, int V) {
  // Shared memory for flagging if any active node in block
  __shared__ bool s_active;

  int tid = threadIdx.x + blockIdx.x * blockDim.x;

  // First thread checks if any node in this block is active
  if (threadIdx.x == 0) {
    s_active = false;
  }
  __syncthreads();

  // Check if this is an active node
  if (tid < V && d_updated[tid] == 1) {
    s_active = true;
  }
  __syncthreads();

  // Early exit if no active nodes in this block
  if (!s_active)
    return;

  // Process only if thread is valid and node was updated
  if (tid < V && d_updated[tid] == 1) {
    // Reset update flag
    d_updated[tid] = 0;

    // Cache node distance to avoid repeated global memory loads
    int node_dist = d_dist[tid];

    // Process all edges for this node
    for (int i = d_offset[tid]; i < d_offset[tid + 1]; i++) {
      int neighbor = d_edges[i];
      int weight = d_weights[i];

      if (node_dist != INF && node_dist + weight < d_dist[neighbor]) {
        atomicMin(&d_dist[neighbor], node_dist + weight);
        d_prev[neighbor] = tid; // Record the predecessor
        d_updated[neighbor] = 1;
      }
    }
  }
}

// CUDA Kernel for counting relaxations with shared memory reduction
__global__ void countUpdates(int *d_updated, int *d_count, int V) {
  __shared__ int s_counts[BLOCK_SIZE];

  int tid = threadIdx.x + blockIdx.x * blockDim.x;
  int lid = threadIdx.x;

  // Initialize shared memory
  s_counts[lid] = 0;

  if (tid < V && d_updated[tid] == 1) {
    s_counts[lid] = 1;
  }

  __syncthreads();

  // Parallel reduction to sum counts within block
  for (int stride = BLOCK_SIZE / 2; stride > 0; stride >>= 1) {
    if (lid < stride) {
      s_counts[lid] += s_counts[lid + stride];
    }
    __syncthreads();
  }

  // First thread in block updates the global counter
  if (lid == 0 && s_counts[0] > 0) {
    atomicAdd(d_count, s_counts[0]);
  }
}

// Function to print spinner animation
void animateProgress() {
  int frame = 0;

  while (processing.load()) {
    cout << "\r[";
    for (int i = 0; i < 50; i++) {
      if (i < progress.load() / 2) {
        cout << "=";
      } else if (i == progress.load() / 2) {
        cout << ">";
      } else {
        cout << " ";
      }
    }

    cout << "] " << progress.load() << "% " << spinnerFrames[frame] << " ";
    cout.flush();

    frame = (frame + 1) % spinnerFrames.size();
    this_thread::sleep_for(chrono::milliseconds(80));
  }

  // Final progress bar
  cout << "\r[";
  for (int i = 0; i < 50; i++) {
    cout << "=";
  }
  cout << "] 100% ✓" << endl;
}

// Sequential Dijkstra with path tracking
vector<int> dijkstraSequential(int src, vector<vector<pair<int, int>>> &adj,
                               int V, vector<int> &prev) {
  vector<int> dist(V, INF);
  prev.resize(V, -1);
  priority_queue<pair<int, int>, vector<pair<int, int>>, greater<>> pq;
  dist[src] = 0;
  pq.emplace(0, src);

  int processedNodes = 0;

  while (!pq.empty()) {
    auto [d, u] = pq.top();
    pq.pop();
    if (d > dist[u])
      continue;

    processedNodes++;
    if (processedNodes % (V / 100 + 1) == 0) {
      progress.store(min(99, (processedNodes * 100) / V));
    }

    for (auto &[v, w] : adj[u]) {
      if (dist[u] + w < dist[v]) {
        dist[v] = dist[u] + w;
        prev[v] = u; // Store predecessor
        pq.emplace(dist[v], v);
      }
    }
  }

  return dist;
}

// Function to reconstruct path from source to target
vector<int> reconstructPath(const vector<int> &prev, int target) {
  vector<int> path;
  for (int at = target; at != -1; at = prev[at]) {
    path.push_back(at);
  }
  reverse(path.begin(), path.end());
  return path;
}

// Function to print path with edge weights in 1-(10)>2 format
void printPath(const vector<int> &path, vector<vector<pair<int, int>>> &adj,
               const vector<int> &dist) {
  if (path.empty()) {
    cout << "No path exists!" << endl;
    return;
  }

  cout << "\n========================================\n";
  cout << "     Shortest Path with Weights\n";
  cout << "========================================\n";

  cout << "Path: ";
  for (size_t i = 0; i < path.size(); i++) {
    cout << path[i];
    if (i < path.size() - 1) {
      // Find edge weight between path[i] and path[i+1]
      int u = path[i];
      int v = path[i + 1];
      int weight = -1;
      for (auto &p : adj[u]) {
        if (p.first == v) {
          weight = p.second;
          break;
        }
      }
      cout << "-(" << weight << ")>";
    }
  }
  cout << "\nTotal Distance: " << dist[path.back()] << endl;
}

void readGraphFlat(const string &filename, vector<Edge> &edges, int &V,
                   int &E) {
  ifstream infile(filename);
  int u, v, w;
  V = 0;
  E = 0;

  int totalBytes = 0;
  infile.seekg(0, ios::end);
  totalBytes = infile.tellg();
  infile.seekg(0, ios::beg);

  long long bytesRead = 0;
  int lastProgress = 0;

  while (infile >> u >> v >> w) {
    edges.push_back({u, v, w});
    V = max(V, max(u, v));
    E++;

    bytesRead = infile.tellg();
    int currentProgress =
        min(99, static_cast<int>(bytesRead * 100 / totalBytes));
    if (currentProgress > lastProgress) {
      progress.store(currentProgress);
      lastProgress = currentProgress;
    }

    if (E % 1000000 == 0) {
      cout << "\r" << E << " edges read... (" << currentProgress << "%)     ";
      cout.flush();
    }
  }
  V++;
  cout << "\r" << E << " edges read... (100%)     " << endl;
}

// Visualize the graph using ASCII art
void visualizeGraph(const vector<int> &dist, const vector<int> &prev, int V,
                    int target) {
  // Find the path to target
  vector<int> path = reconstructPath(prev, target);

  cout << "\n========================================\n";
  cout << "     Graph Visualization (ASCII)\n";
  cout << "========================================\n";

  // Create an ASCII representation of the graph focusing on the path
  const int width = 80;
  const int height = 20;

  vector<vector<char>> grid(height, vector<char>(width, ' '));

  // Place the nodes in the grid
  for (int i = 0; i < min(V, 50); i++) {
    int x = i % width;
    int y = i / width % height;
    grid[y][x] = (dist[i] < INF) ? 'o' : '.';
  }

  // Mark the path nodes
  for (int node : path) {
    if (node < 50) { // Only show first 50 nodes
      int x = node % width;
      int y = node / width % height;
      grid[y][x] = '*';
    }
  }

  // Display the grid
  for (const auto &row : grid) {
    for (char c : row) {
      cout << c;
    }
    cout << endl;
  }

  cout << "\nLegend: '*' = path node, 'o' = reachable node, '.' = unreachable "
          "node\n";
}

int main(int argc, char *argv[]) {
  if (argc < 2) {
    cout << "Usage: ./dijkstra <input_graph_file> [target_node]\n";
    return 1;
  }

  string filename = argv[1];
  int target = (argc > 2) ? atoi(argv[2]) : 999999; // Default target node

  vector<Edge> edges;
  int V, E;

  cout << "\n========================================\n";
  cout << "     Reading Graph\n";
  cout << "========================================\n";

  // Start progress animation thread for graph reading
  thread animationThread(animateProgress);
  progress.store(0);

  readGraphFlat(filename, edges, V, E);

  // Build adjacency list for sequential
  vector<vector<pair<int, int>>> adj(V);
  for (const auto &e : edges) {
    adj[e.u].emplace_back(e.v, e.w);
  }

  cout << "\nGraph loaded with " << V << " nodes and " << E << " edges.\n";

  cout << "\n========================================\n";
  cout << "     Running Sequential Dijkstra\n";
  cout << "========================================\n";

  // Reset progress for sequential algorithm
  progress.store(0);

  vector<int> prev_seq;
  auto start_seq = chrono::high_resolution_clock::now();
  vector<int> dist_seq = dijkstraSequential(0, adj, V, prev_seq);
  auto end_seq = chrono::high_resolution_clock::now();
  double time_seq =
      chrono::duration<double, milli>(end_seq - start_seq).count();
  // Print sequential path and weights if target is valid
  if (target < V) {
    cout << "\n========================================\n";
    cout << "     Sequential Path to Target Node " << target << "\n";
    cout << "========================================\n";

    if (dist_seq[target] == INF) {
      cout << "No path exists to node " << target << endl;
    } else {
      cout << "Distance: " << dist_seq[target] << endl;
      vector<int> path_seq = reconstructPath(prev_seq, target);

      cout << "Path: ";
      for (size_t i = 0; i < path_seq.size(); i++) {
        cout << path_seq[i];
        if (i < path_seq.size() - 1)
          cout << " -> ";
      }
      cout << endl;

      printPath(path_seq, adj, dist_seq);
    }
  }

  // CSR-like format for CUDA
  vector<int> h_offset(V + 1), h_edges, h_weights;
  int edge_count = 0;
  for (int i = 0; i < V; i++) {
    h_offset[i] = edge_count;
    for (auto &[v, w] : adj[i]) {
      h_edges.push_back(v);
      h_weights.push_back(w);
      edge_count++;
    }
  }
  h_offset[V] = edge_count;

  int *d_edges, *d_weights, *d_offset, *d_dist, *d_updated, *d_prev, *d_count;

  cudaMalloc(&d_edges, h_edges.size() * sizeof(int));
  cudaMalloc(&d_weights, h_weights.size() * sizeof(int));
  cudaMalloc(&d_offset, (V + 1) * sizeof(int));
  cudaMalloc(&d_dist, V * sizeof(int));
  cudaMalloc(&d_updated, V * sizeof(int));
  cudaMalloc(&d_prev, V * sizeof(int));
  cudaMalloc(&d_count, sizeof(int));

  vector<int> h_dist(V, INF);
  h_dist[0] = 0;
  vector<int> h_updated(V, 0);
  h_updated[0] = 1;
  vector<int> h_prev(V, -1);
  int h_count = 0;

  cudaMemcpy(d_edges, h_edges.data(), h_edges.size() * sizeof(int),
             cudaMemcpyHostToDevice);
  cudaMemcpy(d_weights, h_weights.data(), h_weights.size() * sizeof(int),
             cudaMemcpyHostToDevice);
  cudaMemcpy(d_offset, h_offset.data(), (V + 1) * sizeof(int),
             cudaMemcpyHostToDevice);
  cudaMemcpy(d_dist, h_dist.data(), V * sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_updated, h_updated.data(), V * sizeof(int),
             cudaMemcpyHostToDevice);
  cudaMemcpy(d_prev, h_prev.data(), V * sizeof(int), cudaMemcpyHostToDevice);

  cout << "\n========================================\n";
  cout << "     Running Parallel CUDA Relaxation\n";
  cout << "     (With Shared Memory Optimization)\n";
  cout << "========================================\n";

  // Reset progress for parallel algorithm
  progress.store(0);

  auto start_par = chrono::high_resolution_clock::now();

  int iterations = 0;
  bool hasUpdates = true;

  while (hasUpdates && iterations < V - 1) {
    iterations++;

    // Zero the count
    h_count = 0;
    cudaMemcpy(d_count, &h_count, sizeof(int), cudaMemcpyHostToDevice);

    // Launch relaxation kernel with shared memory
    int numBlocks = (V + BLOCK_SIZE - 1) / BLOCK_SIZE;
    relaxEdges<<<numBlocks, BLOCK_SIZE>>>(d_dist, d_edges, d_weights, d_offset,
                                          d_updated, d_prev, V);
    cudaDeviceSynchronize();

    // Count updates with shared memory reduction
    countUpdates<<<numBlocks, BLOCK_SIZE>>>(d_updated, d_count, V);
    cudaDeviceSynchronize();

    // Check if we have updates
    cudaMemcpy(&h_count, d_count, sizeof(int), cudaMemcpyDeviceToHost);
    hasUpdates = (h_count > 0);

    // Update progress
    progress.store(min(99, (iterations * 100) / V));
  }

  auto end_par = chrono::high_resolution_clock::now();
  double time_par =
      chrono::duration<double, milli>(end_par - start_par).count();

  cudaMemcpy(h_dist.data(), d_dist, V * sizeof(int), cudaMemcpyDeviceToHost);
  cudaMemcpy(h_prev.data(), d_prev, V * sizeof(int), cudaMemcpyDeviceToHost);

  // Signal animation thread to stop
  processing.store(false);
  animationThread.join();

  cout << "\n========================================\n";
  cout << "     Results\n";
  cout << "========================================\n";
  cout << "Sequential Time: " << time_seq << " ms\n";
  cout << "Parallel Time:   " << time_par << " ms\n";
  cout << "Speedup:         " << time_seq / time_par << "x\n";
  cout << "CUDA Iterations: " << iterations << endl;

  // Verify results match
  bool match = true;
  int firstMismatch = -1;
  for (int i = 0; i < min(V, 1000); i++) { // Check first 1000 nodes
    if (dist_seq[i] != h_dist[i] && !(dist_seq[i] == INF && h_dist[i] == INF)) {
      if (firstMismatch == -1)
        firstMismatch = i;
      match = false;
      // Don't break immediately, continue checking to find how many mismatches
      if (i < 10) { // Print first few mismatches
        cout << "Mismatch at node " << i << ": Sequential = " << dist_seq[i]
             << ", Parallel = " << h_dist[i] << endl;
      }
    }
  }

  if (match) {
    cout << "Results match! ✓\n";
  } else {
    int mismatchCount = 0;
    for (int i = 0; i < min(V, 1000); i++) {
      if (dist_seq[i] != h_dist[i] &&
          !(dist_seq[i] == INF && h_dist[i] == INF)) {
        mismatchCount++;
      }
    }
    cout << "Results differ! ✗ (Found " << mismatchCount
         << " mismatches in first 1000 nodes)\n";
    cout << "First mismatch at node " << firstMismatch << endl;

    // Print some analysis of the mismatches
    if (firstMismatch >= 0) {
      cout << "\nAnalyzing mismatch at node " << firstMismatch << ":\n";
      cout << "Sequential distance: " << dist_seq[firstMismatch] << "\n";
      cout << "Parallel distance: " << h_dist[firstMismatch] << "\n";

      // Print predecessor analysis
      int seq_pred = prev_seq[firstMismatch];
      int par_pred = h_prev[firstMismatch];
      cout << "Sequential predecessor: " << seq_pred
           << " (distance: " << (seq_pred >= 0 ? dist_seq[seq_pred] : -1)
           << ")\n";
      cout << "Parallel predecessor: " << par_pred
           << " (distance: " << (par_pred >= 0 ? h_dist[par_pred] : -1)
           << ")\n";
    }
  }

  // Print path to target node
  if (target < V) {
    cout << "\n========================================\n";
    cout << "     Path to Target Node " << target << "\n";
    cout << "========================================\n";

    if (h_dist[target] == INF) {
      cout << "No path exists to node " << target << endl;
    } else {
      cout << "Distance: " << h_dist[target] << endl;

      // Reconstruct path from source to target
      vector<int> path = reconstructPath(h_prev, target);

      cout << "Path: ";
      for (size_t i = 0; i < path.size(); i++) {
        cout << path[i];
        if (i < path.size() - 1)
          cout << " -> ";
      }
      cout << endl;

      // Detailed path with weights
      printPath(path, adj, h_dist);

      // ASCII visualization of the graph
      visualizeGraph(h_dist, h_prev, V, target);
    }
  }

  cudaFree(d_edges);
  cudaFree(d_weights);
  cudaFree(d_offset);
  cudaFree(d_dist);
  cudaFree(d_updated);
  cudaFree(d_prev);
  cudaFree(d_count);

  return 0;
}

Writing /kaggle/working/tp.cu


In [25]:
!nvcc /kaggle/working/tp.cu -o /kaggle/working/tp

In [27]:
!/kaggle/working/tp /kaggle/working/graph_10M.txt 99999


     Reading Graph
20000000 edges read... (100%)     ================>] 99% | 
[=================================================>] 99% - 
Graph loaded with 10000000 nodes and 20000000 edges.

     Running Sequential Dijkstra
[=================================================>] 99% \ 
     Sequential Path to Target Node 99999
Distance: 202
Path: 0 -> 545833 -> 3777415 -> 3777416 -> 6519681 -> 6519682 -> 6519683 -> 6519684 -> 5799790 -> 3568945 -> 7168348 -> 7168349 -> 8758370 -> 8758371 -> 7580466 -> 8767714 -> 8767715 -> 8767716 -> 4496393 -> 9238910 -> 9895746 -> 6493178 -> 6426114 -> 6426115 -> 6426116 -> 6426117 -> 9932408 -> 9932409 -> 9932410 -> 99998 -> 99999

     Shortest Path with Weights
Path: 0-(2)>545833-(3)>3777415-(6)>3777416-(5)>6519681-(7)>6519682-(3)>6519683-(5)>6519684-(2)>5799790-(3)>3568945-(4)>7168348-(15)>7168349-(4)>8758370-(9)>8758371-(6)>7580466-(9)>8767714-(15)>8767715-(11)>8767716-(3)>4496393-(12)>9238910-(9)>9895746-(7)>6493178-(4)>6426114-(5)>6426115-(1)>